<a href="https://colab.research.google.com/github/roitraining/SparkforDataEngineers/blob/Development/Ch03_SparkSQL_Lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
sys.path.append('/home/student/ROI/Spark')
from initspark import *
sc, spark, conf = initspark()

### *LAB:* Read the northwind JSON products and make it into a TempView and do the same with the CSVHeaders version of categories.

In [2]:
categories = spark.read.csv('/home/student/ROI/Spark/datasets/northwind/CSVHeaders/categories', header=True, inferSchema = True)
print(categories)
categories.show(10)
categories.createOrReplaceTempView('categories') 


DataFrame[CategoryID: int, CategoryName: string, Description: string]
+----------+--------------+--------------------+
|CategoryID|  CategoryName|         Description|
+----------+--------------+--------------------+
|         1|     Beverages|Soft drinks coffe...|
|         2|    Condiments|Sweet and savory ...|
|         3|   Confections|Desserts candies ...|
|         4|Dairy Products|             Cheeses|
|         5|Grains/Cereals|Breads crackers p...|
|         6|  Meat/Poultry|      Prepared meats|
|         7|       Produce|Dried fruit and b...|
|         8|       Seafood|    Seaweed and fish|
+----------+--------------+--------------------+



In [3]:
products = spark.read.json('/home/student/ROI/Spark/datasets/northwind/JSON/products')
print(products)
products.show()
products.createOrReplaceTempView('products') 


DataFrame[categoryid: bigint, discontinued: bigint, productid: bigint, productname: string, quantityperunit: string, reorderlevel: bigint, supplierid: bigint, unitprice: double, unitsinstock: bigint, unitsonorder: bigint]
+----------+------------+---------+--------------------+--------------------+------------+----------+---------+------------+------------+
|categoryid|discontinued|productid|         productname|     quantityperunit|reorderlevel|supplierid|unitprice|unitsinstock|unitsonorder|
+----------+------------+---------+--------------------+--------------------+------------+----------+---------+------------+------------+
|         1|           1|        1|                Chai|  10 boxes x 30 bags|          10|         8|     18.0|          39|           0|
|         1|           1|        2|               Chang|  24 - 12 oz bottles|          25|         1|     19.0|          17|          40|
|         2|           0|        3|       Aniseed Syrup| 12 - 550 ml bottles|          2

In [6]:
sql = """
select *
from products as p
join categories as c on p.categoryid = c.categoryid
"""

spark.sql(sql).drop('Description').show()

+----------+------------+---------+--------------------+--------------------+------------+----------+---------+------------+------------+----------+--------------+
|categoryid|discontinued|productid|         productname|     quantityperunit|reorderlevel|supplierid|unitprice|unitsinstock|unitsonorder|CategoryID|  CategoryName|
+----------+------------+---------+--------------------+--------------------+------------+----------+---------+------------+------------+----------+--------------+
|         1|           1|        1|                Chai|  10 boxes x 30 bags|          10|         8|     18.0|          39|           0|         1|     Beverages|
|         1|           1|        2|               Chang|  24 - 12 oz bottles|          25|         1|     19.0|          17|          40|         1|     Beverages|
|         2|           0|        3|       Aniseed Syrup| 12 - 550 ml bottles|          25|         1|     10.0|          13|          70|         2|    Condiments|
|         2|    

In [9]:
sql = """
select c.CategoryID, c.CategoryName, ProductID, ProductName, UnitPrice, UnitsInStock
, UnitPrice * UnitsInStock as Value 
from products as p
join categories as c on p.categoryid = c.categoryid
where UnitPrice * UnitsInStock > 1000
"""

spark.sql(sql).show()

+----------+--------------+---------+--------------------+---------+------------+------------------+
|CategoryID|  CategoryName|ProductID|         ProductName|UnitPrice|UnitsInStock|             Value|
+----------+--------------+---------+--------------------+---------+------------+------------------+
|         2|    Condiments|        4|Chef Anton's Caju...|     22.0|          53|            1166.0|
|         2|    Condiments|        6|Grandma's Boysenb...|     25.0|         120|            3000.0|
|         6|  Meat/Poultry|        9|     Mishi Kobe Niku|     97.0|          29|            2813.0|
|         4|Dairy Products|       12|Queso Manchego La...|     38.0|          86|            3268.0|
|         8|       Seafood|       18|    Carnarvon Tigers|     62.5|          42|            2625.0|
|         3|   Confections|       20|Sir Rodney's Marm...|     81.0|          40|            3240.0|
|         5|Grains/Cereals|       22| Gustaf's Knackebrod|     21.0|         104|          

In [11]:
sql = """
select * from 
(
select c.CategoryID, c.CategoryName, ProductID, ProductName, UnitPrice, UnitsInStock
, UnitPrice * UnitsInStock as Value 
from products as p
join categories as c on p.categoryid = c.categoryid ) as t
where UnitPrice * UnitsInStock > 1000
"""

spark.sql(sql).show()

+----------+--------------+---------+--------------------+---------+------------+------------------+
|CategoryID|  CategoryName|ProductID|         ProductName|UnitPrice|UnitsInStock|             Value|
+----------+--------------+---------+--------------------+---------+------------+------------------+
|         2|    Condiments|        4|Chef Anton's Caju...|     22.0|          53|            1166.0|
|         2|    Condiments|        6|Grandma's Boysenb...|     25.0|         120|            3000.0|
|         6|  Meat/Poultry|        9|     Mishi Kobe Niku|     97.0|          29|            2813.0|
|         4|Dairy Products|       12|Queso Manchego La...|     38.0|          86|            3268.0|
|         8|       Seafood|       18|    Carnarvon Tigers|     62.5|          42|            2625.0|
|         3|   Confections|       20|Sir Rodney's Marm...|     81.0|          40|            3240.0|
|         5|Grains/Cereals|       22| Gustaf's Knackebrod|     21.0|         104|          

In [10]:
sql = """
select c.CategoryID, c.CategoryName, ProductID, ProductName, UnitPrice, UnitsInStock
, UnitPrice * UnitsInStock as Value 
from products as p
join categories as c on p.categoryid = c.categoryid
"""

spark.sql(sql).where('Value > 1000').show()

+----------+--------------+---------+--------------------+---------+------------+------------------+
|CategoryID|  CategoryName|ProductID|         ProductName|UnitPrice|UnitsInStock|             Value|
+----------+--------------+---------+--------------------+---------+------------+------------------+
|         2|    Condiments|        4|Chef Anton's Caju...|     22.0|          53|            1166.0|
|         2|    Condiments|        6|Grandma's Boysenb...|     25.0|         120|            3000.0|
|         6|  Meat/Poultry|        9|     Mishi Kobe Niku|     97.0|          29|            2813.0|
|         4|Dairy Products|       12|Queso Manchego La...|     38.0|          86|            3268.0|
|         8|       Seafood|       18|    Carnarvon Tigers|     62.5|          42|            2625.0|
|         3|   Confections|       20|Sir Rodney's Marm...|     81.0|          40|            3240.0|
|         5|Grains/Cereals|       22| Gustaf's Knackebrod|     21.0|         104|          